# Week 5 - Pandas and Data Cleanup

#### Recap from Previous Class:
- Fetched data using Yelp API
- Analyzed and processed the data for the Top5 App
- Pulled duration data using Mapbox API
- Learned about the following concepts:
    - Pandas Data Structures
    - DataFrame and Series attributes
    - Selecting and Subsetting in DataFrames (.loc and .iloc)
    - Pass by Reference vs Pass by Values
    - Looping through DataFrame rows
    - Method Chaining
    - Apply method and lambda
    - Mapping values in DataFrames

#### Topics Covered in Videos:
- Missing Values
- Wrangling, Reshaping and Pivot Tables
- Data Analysis: Split-Apply-Combine and Pandas GroupBy
- Pandas str and regex

---

#### Our Focus for today:
1. Understand how to deal with missing values 
    - An in-depth review of recognizing patters and handling missing values in terms of the context
    - Use ML for imputation
2. Learn about dummy variables and their creation
3. Apply the concept of GroupBy-Split-Combine
4. Learn about Pivot, Reshape and Melt
5. Str and Regex in Python
    - Processing Text Data


<img src="./images/app_image.png" width="300">

#### <span style='color:blue'>Task 0: Fetching Yelp Data and Structuring DataFrame</span> 

In [ ]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Extracting Yelp Key
from configparser import ConfigParser
config = ConfigParser()
config.read('config.ini')

API_KEY = config["YELP"]["api-key"]

In [ ]:
# import data from yelp
import requests
import json

base_url = 'https://api.yelp.com/v3'
endpoint = '/businesses/search'

# Construct the full URL for the API request
url = f"{base_url}{endpoint}"
headers = {'Authorization': API_KEY}
params = {
    'location': 'Denver, CO',  # Location to search for businesses
    'limit': 50,               # Limit the number of results to 50
    'term': 'restaurant',      # Search term to look for restaurants
    'radius': 2000             # Search radius in meters
}

try:
    # Make the GET request to the Yelp API
    response = requests.get(url, headers=headers, params=params)
    # If the request is successful, print the results in a formatted JSON
    if response.status_code == 200:
        result = response.json()
    else:
        # Raise an exception if the request was not successful
        response.raise_for_status()
except requests.exceptions.RequestException as e:
    # Print an error message if an exception occurs during the request
    print(f"Error occurred: {e}")

In [ ]:
# For fetching businesses after pagination (set to 50 results per page)
import pandas as pd
restaurant_df = pd.DataFrame(result['businesses'])

# Append data
for i in range(1, 5):
    url_params = {
        'location': 'Denver, CO',
        'limit': 50,
        'term': 'restaurant',
        'radius': 5000,
        'offset': (i*50) # Offset for pagination
    }
    if i == 4:
        url_params['limit'] = 40 # Limit the last page to 40 results
    try:
        # Make the GET request to the Yelp API for additional pages
        response = requests.get(url, headers=headers, params=url_params)
        if response.status_code == 200:
            result = json.loads(response.content)
            data = result['businesses']
            new_df = pd.DataFrame(data)
            restaurant_df = pd.concat([restaurant_df, new_df], ignore_index=True)
        else:
            # Raise an exception if the request was not successful
            response.raise_for_status()
    except requests.exceptions.RequestException as e:
        # Print an error message if an exception occurs during the request
        print(f"Error occurred: {e}")

In [ ]:
# create a deep copy
restaurant_df_clean = restaurant_df.copy(deep=True)

# capatialize columns
restaurant_df_clean.columns = restaurant_df_clean.columns.str.capitalize()

# Subset columns using loc
restaurant_df_clean = restaurant_df_clean.loc[:, ['Id', 'Name', 'Is_closed', 'Review_count', 'Categories', 
                                      'Rating', 'Coordinates', 'Transactions', 'Price', 'Location',
                                      'Distance', 'Business_hours']]

In [ ]:
# extract display address from location
restaurant_df_clean['Location'] = restaurant_df_clean['Location'].apply(lambda x: x['display_address'])

In [ ]:
# convert data types
restaurant_df_clean = restaurant_df_clean.astype({
    'Price': 'category',
    'Name': 'string',
    'Id':'string'
})

def alter_transactions(df):
    for transaction in ['delivery', 'pickup', 'restaurant_reservation']:
        df.loc[:, transaction] = 0
        df.loc[:, transaction] = df.loc[:, 'Transactions'].apply(lambda x: 1 if transaction in x else 0)

alter_transactions(restaurant_df_clean)

restaurant_df_clean.head(3)

### Dealing with Missing Values

Understanding the nature of missing data is crucial for choosing appropriate handling methods, such as imputation techniques or adjustments in statistical models. The interpretation can significantly impact the validity of conclusions drawn from the data.

There are several ways to interpret missing values in data science, each with different implications for analysis and modeling. Here are some common interpretations:

1. **Missing Completely at Random (MCAR)**:
   - Data is missing due to entirely random factors
   - No relationship between the missing data and other variables
   - Least problematic for analysis. Safe to remove MCAR data as it does not introduce bias into the analysis
   - E.g: Data missing due to technical issues or equipment malfunctions.

2. **Missing at Random (MAR)**:
   - Missing data is related to other observed variables but not to the missing variable itself
   - Requires careful handling to avoid bias. Techniques like imputation or weighting can be used
   - E.g. Data missing due to known characteristics of the population, such as non-response in surveys.

3. **Missing Not at Random (MNAR)**:
   - Missing data is related to the unobserved values themselves
   - Most challenging to handle and can lead to biased results
   - Requires specialized techniques like multiple imputation, sensitivity analysis or selection models to avoid bias.
   - E.g. Data missing due to the nature of the variable (e.g., sensitive questions in surveys)

<img src="./images/missing_vals.png" width="400">

4. Structural missingness:
   - Data is missing due to the inherent structure of the data collection process
   - Example: Questions that are only asked if a previous question is answered in a certain way

5. Truncated data:
   - Values beyond a certain threshold are completely unobserved

6. Intermittent missingness:
   - Data is missing for some time points in a time series
   - Common in longitudinal studies

7. Dropout:
   - Subjects leave a study before its completion, leading to missing data in later time points

8. Informative missingness:
   - The fact that data is missing provides information about the underlying phenomenon


Additional Readings:
- https://www.geeksforgeeks.org/ml-handling-missing-values/
- https://www.freecodecamp.org/news/how-to-handle-missing-data-in-a-dataset/
- https://www.mastersindatascience.org/learning/how-to-deal-with-missing-data/
- https://www.linkedin.com/advice/3/what-some-common-causes-types-missing-values-datasets
- https://www.linkedin.com/advice/0/what-best-practices-identifying-handling-missing

#### <span style='color:blue'>Task 1: Handle Missing Price Values</span> 

In [ ]:
# value count with missing values
restaurant_df_clean.loc[:, 'Price'].value_counts(dropna=False)

In [ ]:
restaurant_df_imputed = restaurant_df_clean.copy(deep=True)
# fill missing values with mode
restaurant_df_imputed.loc[:, 'Price'].fillna(restaurant_df_imputed.loc[:, 'Price'].mode()[0], inplace=False).value_counts(dropna=False)


Let's try imputation with Random Forest.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

# Prepare the data
X = restaurant_df_imputed.loc[:, ['Rating', 'Review_count', 'Price']]
y = restaurant_df_imputed['Price']

# Do label encoding
le = LabelEncoder()
y_train = le.fit_transform(y.dropna())

In [ ]:
X_train = X.loc[X['Price'].notnull(), ['Rating', 'Review_count']]
X_test = X.loc[X['Price'].isnull(), ['Rating', 'Review_count']]

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

In [ ]:
missing_price = restaurant_df_imputed['Price'].isnull()

# Impute missing values
restaurant_df_imputed.loc[missing_price, 'Price'] = \
le.inverse_transform(
    rf_model.predict(
        restaurant_df_imputed.loc[missing_price, ['Rating', 'Review_count']]
        )
)

In [ ]:
# value counts
restaurant_df_imputed.loc[:, 'Price'].value_counts(dropna=False)

---

#### Dummy Variables

- Representing categorical data: They allow us to include categorical variables in Machine Learning models that require numerical inputs.
- Avoiding ordinal relationships: Dummy coding prevents implying an ordinal relationship where none exists.
- Improved interpretability: Each dummy variable has a clear interpretation in the context of the model.

#### <span style='color:blue'>Task 2: Create dummy for categories</span> 

In [ ]:
# check categories value
restaurant_df_imputed.loc[51, 'Categories']

In [ ]:
# in each row, create a category list
restaurant_df_imputed['Categories'] = restaurant_df_imputed['Categories'].apply(lambda x: [category['title'] for category in x])

In [ ]:
# loop through categories using itertuples
category_set = dict()
for row in restaurant_df_imputed.itertuples():
    for category in row.Categories:
        category_set[category] = category_set.get(category, 0) + 1


In [ ]:
# take top 10
category_set = dict(sorted(category_set.items(), key=lambda x: x[1], reverse=True)[:10])

In [ ]:
# display category_set
category_set

In [ ]:
# create dummy variables
for category in category_set:
    restaurant_df_imputed[category] = restaurant_df_imputed['Categories'].apply(lambda x: 1 if category in x else 0)

In [ ]:
restaurant_df_imputed.head(3)

We can also use pd.get_dummies for simpler ways of creating dummies for categorical variables.

In [ ]:
# create dummy variables for Price column
pd.get_dummies(restaurant_df_imputed, columns=['Price']).head(3)

### Groupby-Split-Combine

- Group: Divide your data into groups based on some common characteristic.
- Split: Separate the data into these groups.
- Apply: Perform an operation or calculation on each group independently.

<img src="./images/groupby.png" width="600">


A list of all the functions can be found in the [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#grouping-operations)


| Function          | Description                                                           |
|-------------------|-----------------------------------------------------------------------|
| **Aggregation Functions** |
| `sum()`           | Compute the sum of group values.                                      |
| `mean()`          | Compute the mean of group values.                                     |
| `median()`        | Compute the median of group values.                                   |
| `min()`           | Compute the minimum of group values.                                  |
| `max()`           | Compute the maximum of group values.                                  |
| `count()`         | Compute the count of group values.                                    |
| `size()`          | Compute the size of each group.                                       |
| `std()`           | Compute the standard deviation of group values.                       |
| `var()`           | Compute the variance of group values.                                 |
| `sem()`           | Compute the standard error of the mean of group values.               |
| `first()`         | Compute the first value of each group.                                |
| `last()`          | Compute the last value of each group.                                 |
| `nth(n)`          | Compute the nth value of each group.                                  |
| `prod()`          | Compute the product of group values.                                  |
| `any()`           | Returns True if any element in the group is True.                     |
| `all()`           | Returns True if all elements in the group are True.                   |
| **Transformation Functions** |
| `transform(func)` | Apply a function to each group and return a DataFrame with the same shape as the original. |
| **Apply Functions** |
| `apply(func)`     | Apply a function to each group and combine the results into a DataFrame, Series, or scalar depending on the function's output. |
| **Descriptive Statistics** |
| `describe()`      | Generate descriptive statistics for each group.                       |
| `quantile(q)`     | Compute the quantile of each group.                                   |
| `mad()`           | Compute the mean absolute deviation of group values.                  |
| `skew()`          | Compute the skewness of group values.                                 |
| `kurt()`          | Compute the kurtosis of group values.                                 |
| **Other Functions** |
| `agg(func)`       | Aggregate using one or more operations over the specified axis.       |
| `cumsum()`        | Compute the cumulative sum of group values.                           |
| `cumprod()`       | Compute the cumulative product of group values.                       |
| `cummin()`        | Compute the cumulative minimum of group values.                       |
| `cummax()`        | Compute the cumulative maximum of group values.                       |
| `head(n)`         | Return the first `n` rows of each group.                              |
| `tail(n)`         | Return the last `n` rows of each group.                               |
| `ngroup()`        | Number each group from 0 to the number of groups - 1.                 |
| `nth(n)`          | Take the nth row from each group.                                     |

#### <span style='color:blue'>Task 3: Get summarized information</span> 

In [ ]:
restaurant_df_imputed.groupby('Price').count()

In [ ]:
# Groupby price and count and average the ratings and review-count
restaurant_df_imputed.groupby('Price').agg({'Review_count': 'mean', 'Rating': 'mean'})

In [ ]:
# Groupby price and count for all restuarant categories
restaurant_df_imputed.groupby('Price')[list(category_set.keys())].sum()


### Reshaping: Pivot and Melt

**Multi-indexing**: Multi-indexing in pandas allows you to work with more complex data structures by using multiple levels of indexes (row or column labels). This is particularly useful for handling data with multiple dimensions or hierarchical data.

In [ ]:
data = {
    'Month': ['Jan', 'Jan', 'Jan', 'Jan', 'Feb', 'Feb', 'Feb', 'Feb'],
    'Region': ['North', 'North', 'South', 'South', 'North', 'North', 'South', 'South'],
    'Product': ['A', 'B', 'A', 'B', 'A', 'B', 'A', 'B'],
    'Sales': [100, 150, 200, 250, 110, 160, 210, 260]
}

df = pd.DataFrame(data)
df.set_index(['Month', 'Region'], inplace=True)
print(df)


**Pivot**: Pivoting is the process of transforming data from a long format to a wide format. In a wide format, we use unique values from a column to create new columns. In other words, spread rows into columns.

<img src="./images/pivot.png" width="600">

**Melt**: Melting is the opposite of pivoting. It transforms data from a wide format to a long format. In a long format, we have more rows and fewer columns. In other words, it gathers columns into rows.

<img src="./images/melt.png" width="600">

#### <span style='color:blue'>Task 4: Create top 5 table for each category</span> 

In [ ]:
# Filter where Mexican is 1 and sortby rating
restaurant_df_imputed.loc[restaurant_df_imputed['Mexican'] == 1, ['Name', 'Review_count', 'Price', 'Rating']].sort_values('Rating', ascending=False).head(5)

In [ ]:
restaurant_df_imputed.columns


In [ ]:
col_indices = [1, 5, 10] + list(range(15, 25))
restaurant_top5 = restaurant_df_imputed.copy(deep=True).iloc[:, col_indices]

In [ ]:
restaurant_top5.head()

In [ ]:
# Melt the DataFrame to have a single category column
melted = restaurant_top5.melt(
                id_vars=['Name', 'Rating', 'Distance'], 
                 value_vars= list(category_set.keys()),
                 var_name='Category', value_name='Is_Category')

In [ ]:
melted.head()

In [ ]:
# Filter only rows where Is_Category is 1
melted = melted[melted['Is_Category'] == 1]

In [ ]:
# Remove the Is_Category column
melted = melted.drop(columns=['Is_Category'])

# Sort by Category, then Ratings, and Duration
melted = melted.sort_values(by=['Category', 'Rating', 'Distance'], ascending=[True, False, True])


In [ ]:
melted.head(10)

In [ ]:

# Group by Category and take top 5
top_5 = melted.groupby('Category').head(5)


In [ ]:
top_5

In [ ]:

# Create a ranking within each category
top_5['Rank'] = top_5.groupby('Category').cumcount() + 1
top_5


In [ ]:

# Set Multi-Index
top_5.set_index(['Category', 'Rank'], inplace=True)


In [ ]:
top_5

### In Class Exercise: https://github.com/rahimrasool/tools1_week5/blob/main/In_class_exercise.ipynb

### Str and Regex

Some of the most essential string methods in Pandas:

1. **`str.lower()`**: Converts strings in the Series to lowercase.
2. **`str.upper()`**: Converts strings in the Series to uppercase.
3. **`str.title()`**: Converts strings in the Series to titlecase (each word starts with an uppercase letter).
4. **`str.capitalize()`**: Capitalizes the first character of each string in the Series.
5. **`str.strip()`**: Removes leading and trailing whitespace from each string in the Series.
6. **`str.lstrip()`**: Removes leading whitespace from each string in the Series.
7. **`str.rstrip()`**: Removes trailing whitespace from each string in the Series.
8. **`str.replace(old, new)`**: Replaces occurrences of a substring within each string with another substring.
9. **`str.cat(sep='')`**: Concatenates strings in the Series with a specified separator.
10. **`str.contains(pattern)`**: Checks if strings in the Series contain a pattern.
11. **`str.startswith(pattern)`**: Checks if strings in the Series start with a pattern.
12. **`str.endswith(pattern)`**: Checks if strings in the Series end with a pattern.
13. **`str.find(sub)`**: Finds the lowest index of the substring.
14. **`str.len()`**: Computes the length of each string in the Series.
15. **`str.split(pat=None, n=-1, expand=False)`**: Splits strings in the Series by a specified delimiter.
16. **`str.join(sep)`**: Joins lists in the Series into a single string with a specified separator.



In [ ]:
restaurant_df_imputed.Location = restaurant_df_imputed.Location.apply(lambda x: ",".join(x))
restaurant_df_imputed.Location.head(2)

In [ ]:
'1223 E 13thCO 80218 CO'.split()[-1]

In [ ]:
restaurant_df_imputed['Zipcode'] = restaurant_df_imputed.Location.apply(lambda x: x.split()[-1])


In [ ]:
restaurant_df_imputed['Zipcode']

In [ ]:
import re
re.search(r"\b\d{5}\b", '1033 E 17th Ave 80218 CO').group(0)

### DateTime in Pandas

- Temporal data (consisting of date and time stamps) is highly common in data analytics and can be processed to create useful features. 
- Pandas offers the ability to work with time series information in various formats. 
- This enables us to perform several operations on datetime values such as sorting, predicting or categorizing data by certain time periods.

</br>

- Datetime from Python’s datetime library provides the ability for manipulating dates and times. 
- We will work with datetime.datetime type which is a combination of both date and time. 
- Pandas will assign either datetime64[ns] or datetime64[ns, tz] data type to such a format.


In [ ]:
# converting a single scalar value
pd.to_datetime("2010/11/12")

In [ ]:
import numpy as np
dates = pd.date_range(start='2024-01-01', end='2024-01-31')
stores = ['Littleton', 'Aurora', 'Highlands']
products = ['Airmax', 'Airforce', 'Jordans']
sales_data = pd.DataFrame({
    'date': np.random.choice(dates, size=300),
    'store': np.random.choice(stores, size=300),
    'product': np.random.choice(products, size=300),
    'sales': np.random.randint(10, 100, size=300)
})

In [ ]:
sales_data.sample(20)

Every Timestamp has a set of time/date properties or temporal features that can be extracted.

For a Series of type datetime, we can use the .dt accessor to extract these properties. This [table](https://pandas.pydata.org/docs/user_guide/timeseries.html#time-date-components) lists all the properties that can be accessed. We will use it to get the day of the week (to classify if the sale was on a weekend or a weekday). We will assign it to a new column in the DataFrame.

In [ ]:
sales_data.loc[:,'Weekday'] = sales_data.loc[:,'date'].dt.weekday
sales_data.sample(10)

### Dask and RAPIDS